In [9]:
import numpy as np
from sklearn.cluster import SpectralClustering,KMeans
from sklearn.metrics import average_precision_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import matthews_corrcoef,accuracy_score,precision_score,recall_score,confusion_matrix,f1_score
from scipy.stats import pearsonr
import matplotlib.pyplot as plt
import os

In [2]:
# HL: transfer from dataden "/umms-kinfai/duolin/ying/reditools2_candidates/"
hash_candidat={}
hash_candidat['AFG-H1_directRNA']='H1-AFG.candidate_sites.tab'
hash_candidat['AFG-H9_directRNA']='H9-AFG.candidate_sites.tab'
hash_candidat["PGC-H1_directRNA"]='H1-PGC.candidate_sites.tab'
hash_candidat["DE-H1_directRNA"]='H1-DE.candidate_sites.tab'
hash_candidat["DE-H9_directRNA"]='H9-DE.candidate_sites.tab'
hash_candidat["GM12878_directRNA"]='GM12878.candidate_sites.tab'
hash_candidat["H1-hESC_directRNA"]='H1-hESC.candidate_sites.tab'
hash_candidat["H9-hESC_directRNA"]='H9-hESC.candidate_sites.tab'
hash_candidat['HEK293T_DKO_directRNA']='HEK293T_WT.candidate_sites.tab'
hash_candidat["HEK293T_WT_directRNA"]='HEK293T_WT.candidate_sites.tab'
hash_candidat["HEK_WT_pass"]='HEK293T_WT.candidate_sites.tab'


In [3]:
modelist=[
    'hg38_incre5train__HEK293T_WT_directRNA',
    'hg38_incre5train__HEK293T_WT_directRNA_AFG-H1_directRNA',
    'hg38_incre5train__HEK293T_WT_directRNA_H9-hESC_directRNA', #no noncandidate data
'hg38_incre5train__HEK293T_WT_directRNA_AFG-H1_directRNA_DE-H1_directRNA', #no noncandidate data
'hg38_incre5train__HEK293T_WT_directRNA_AFG-H1_directRNA_DE-H1_directRNA_H9-hESC_directRNA', #no data

'hg38_merge9alldata5_noearlystop_run1ep40_run2ep60_negsite_KOnonw5_withLSTM',#second general model
]

modelnames=['HEK293T','HEK293T;H1-PE','HEK293T;H9-hESC','HEK293T;H1-PE;H1-DE','HEK293T;H1-PE;H1-DE;H9-hESC','all']

In [4]:
rediportal = "/nfs/turbo/umms-kinfai/haorli/20240314_ReDD_result_data/figure3a/REDIportal_hg38.txt"
REDIportal_sites={}
input = open(rediportal,'r')
for line in input:
    chr_ = line.split()[0]
    pos_ = line.split()[1]
    REDIportal_sites[chr_+"-"+pos_]=1

In [5]:
using_mean_prediction_score=True
coverage_cutoff= 5
ratio_threshod=0.1
datatype='H1-hESC_directRNA'
balance_neg=True
if using_mean_prediction_score:
   score_index=-1
else:
   score_index=-2
candidatefile="/nfs/turbo/umms-kinfai/haorli/20240314_ReDD_result_data/figure2a/reditools2_candidates/"+hash_candidat[datatype]
true_candidate_ratios={}
input = open(candidatefile,'r')
for line in input:
    chr_ = line.split()[0]
    pos_ = line.split()[1].strip()
    true_candidate_ratios[chr_+"-"+pos_]=float(line.split("\t")[3])

In [6]:
all_true_value_selected = []
all_pred_value_selected = []
# all_coords_list = []
for model in modelist:
    filename1="/nfs/turbo/umms-kinfai/haorli/20240314_ReDD_result_data/figure3d/ReDD_results/"+model+"/"+"test_"+datatype+"_onlycandidate_cov"+"5"+"_ratio0_modcov0.sitelev.bed" #background must use no filter except coverage
    filename2="/nfs/turbo/umms-kinfai/haorli/20240314_ReDD_result_data/figure3d/ReDD_results/"+model+"/"+"test_"+datatype+"_noncandidate_cov"+"5"+"_ratio0_modcov0.sitelev.bed"
    true_value_all=[]
    predict_value_all=[]
    num_candidate=0
    index_neg=[]
    input=open(filename1)
    for line in input:
        readcoverage=int(line.split()[3])
        if readcoverage<coverage_cutoff:
             continue

        chr_ = line.split()[0]
        pos_ = line.split()[1]
        if len(line.split("\t")) == 6:
            predictscore=float(line.split("\t")[score_index]) #last score is mean prediciton score
        else:
            predictscore=float(line.split("\t")[-1]) #for old result there is no mean prediction scores

        if chr_+"-"+pos_ in true_candidate_ratios:
             true_value_all.append(true_candidate_ratios[chr_+"-"+pos_])
        else:
             if chr_+"-"+pos_  in REDIportal_sites:
                  continue  #remove all sites in REDIportal_sites from neg set

             true_value_all.append(0)
             index_neg.append(len(true_value_all)-1)

        predict_value_all.append(predictscore)
        num_candidate+=1

    input=open(filename2)
    num_noncandidate=0
    for line in input:
        readcoverage=int(line.split()[3])
        if readcoverage<coverage_cutoff:
             continue

        chr_ = line.split()[0]
        pos_ = line.split()[1]
        if len(line.split("\t")) == 6:
            predictscore=float(line.split("\t")[score_index])
        else:
            predictscore=float(line.split("\t")[-1])

        if chr_+"-"+pos_ in true_candidate_ratios:
             true_value_all.append(true_candidate_ratios[chr_+"-"+pos_])
        else:
             if chr_+"-"+pos_  in REDIportal_sites:
                  continue  #remove all sites in REDIportal_sites from neg set

             true_value_all.append(0)
             index_neg.append(len(true_value_all)-1)

        predict_value_all.append(predictscore)
        num_noncandidate+=1


    true_value_all=np.asarray(true_value_all)
    predict_value_all=np.asarray(predict_value_all)
    if balance_neg:
        np.random.seed(1234)
        np.random.shuffle(index_neg)
    pos_coverage_ratio = 0.1
    if balance_neg:
        index_pos=np.where(true_value_all > pos_coverage_ratio)[0]
        index_pos_len=len(index_pos)
        select_neg=index_neg[:index_pos_len]
        predict_value_selected=predict_value_all[np.concatenate([index_pos,select_neg])]
        true_value_selected=true_value_all[np.concatenate([index_pos,select_neg])]
    else:
         predict_value_selected=predict_value_all
         true_value_selected=true_value_all

    true_value_selected = (true_value_selected >= pos_coverage_ratio).astype(int) 
    all_true_value_selected.append(true_value_selected)
    all_pred_value_selected.append(predict_value_selected)


In [10]:
model_performance = {}
correct_sites = []
for model,true_value_selected,prediction_score in zip(modelnames,all_true_value_selected,all_pred_value_selected):
#     print(model)
    # determine FPR == 0.05 from ROC
    fpr_curve, tpr_curve, thresholds = roc_curve(true_value_selected, prediction_score,drop_intermediate=False)
    ratio_threshod = thresholds[np.argmin(np.absolute(fpr_curve-0.05))]
    predict_value_selected = (prediction_score>=ratio_threshod).astype(int)
    
    predictions = predict_value_selected
    correct_roc_thres = predictions == true_value_selected
    correct_sites.append([model,correct_roc_thres])
    
    tn, fp, fn, tp = confusion_matrix(true_value_selected, predict_value_selected).ravel()
    tpr = tp /(tp+fn)
    fpr = fp /(fp+tn)
    acc = (tp+tn)/(tp+tn+fp+fn)
    precision = precision_score(true_value_selected, predict_value_selected)
    recall = recall_score(true_value_selected, predict_value_selected)
    f1 = f1_score(true_value_selected, predict_value_selected)

    model_performance[model] = [tpr,fpr,precision,recall,acc,f1,ratio_threshod]

In [50]:
df = pd.DataFrame(model_performance).T
df.index.name ='Model'

df.columns=['TPR','FPR','Precision','Recall','Accuracy','F1-score','ratio_threshod']

In [51]:
df.to_csv('plot_data/site_level_metrics.tsv',sep='\t')

In [13]:
import statsmodels.stats.contingency_tables
import pandas as pd
def cal_contigency_table(rowA,rowB):
    roc_contigency = [[len(np.where((rowA['correct_ROC_threshold'] == True) & (rowB['correct_ROC_threshold']==True))[0]),
    len(np.where((rowA['correct_ROC_threshold'] == True) & (rowB['correct_ROC_threshold']==False))[0])],
    [len(np.where((rowA['correct_ROC_threshold'] == False) & (rowB['correct_ROC_threshold']==True))[0]),
    len(np.where((rowA['correct_ROC_threshold'] == False) & (rowB['correct_ROC_threshold']==False))[0])]]
    
    return roc_contigency
correct_df = pd.DataFrame(correct_sites,columns=['Method','correct_ROC_threshold'])

In [14]:
correct_df

,Method,correct_ROC_threshold
0,HEK293T,"[False, False, False, False, True, True, True,..."
1,HEK293T;H1-PE,"[False, False, True, False, True, False, True,..."
2,HEK293T;H9-hESC,"[False, True, True, False, True, False, True, ..."
3,HEK293T;H1-PE;H1-DE,"[True, False, True, True, False, True, True, F..."
4,HEK293T;H1-PE;H1-DE;H9-hESC,"[True, False, True, False, True, False, True, ..."
5,all,"[True, False, True, False, True, True, True, F..."


In [17]:
for met,i in zip(['HEK293T;H1-PE','HEK293T;H9-hESC','HEK293T;H1-PE;H1-DE'],[1,2,3,4]):
    print(met)
    roc_contigency = cal_contigency_table(correct_df.iloc[0],correct_df.iloc[i])
    print(statsmodels.stats.contingency_tables.mcnemar(roc_contigency, exact=False, correction=False).pvalue)

HEK293T;H1-PE
5.358051166961491e-07
HEK293T;H9-hESC
5.296360162027567e-19
HEK293T;H1-PE;H1-DE
6.918626623748691e-17
